In [ ]:
!pip install plotly

In [3]:
%%time

import os
import boto3
import re
import sagemaker
import pandas as pd

from multiprocessing import Pool
from time import sleep


role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
bucket = sagemaker.Session().default_bucket()
# customize to your bucket where you have stored the data
bucket_path = "https://s3.console.aws.amazon.com/s3/buckets/robi-datathon-dataset-2022?region=ap-southeast-1&tab=objects".format(region, bucket)


CPU times: user 1.02 s, sys: 533 ms, total: 1.55 s
Wall time: 990 ms


In [4]:
bucket

'sagemaker-us-east-1-191805398547'

In [5]:
bucket = 'robi-datathon-dataset-2022/data/transaction'
data_key = 'part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

In [6]:
data_location

's3://robi-datathon-dataset-2022/data/transaction/part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'

## **Visualizing all the *Retail Points* in Bangladesh**

Dropped all the points that were out side of the maximum rectangle that bounds Bangladesh

In [ ]:
retail_points = df_agent.drop(columns=['ts']).drop_duplicates()

In [ ]:
retail_points.info()

In [ ]:
retail_points['agent'].nunique()

In [ ]:
retail_points.shape

In [ ]:
import json
bd_dist = json.load(open("bd.json", 'r'))

In [ ]:
import plotly.express as px

fig = px.scatter_geo(retail_points,
    lat='lat',
    lon='lon',
    color='transactions',
    geojson=bd_dist,
    opacity=1,
    size_max=5,
    width=1080,
    height=1920,
    scope='asia',
    hover_name='transactions'
)
fig.update_layout(title = 'Retail points', title_x=0.5)
fig.update_geos(fitbounds='locations', visible=False)
fig.show()

## **What is peak hour in terms of number of transactions?**

**Proposed**: group by hour each transaction dataset > created (key, value) pair where key is hour and value is the number of transaction > Aggregated all the counts based on the key for all transactions > Got the hour (key) with maximum aggregated count of transactions

**Assumption**: Each slice has consecutive samples

In [14]:
map_hour = [0.0]*24

for i in range(0, 10):
    data_key = f'part-0000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    print(f'Reading {data_key}')
    df = pd.read_csv(data_location)
    print(f'Read {data_key}')
    df = df.dropna()
    print(f'Dropped Nones')
    df['ts'] = pd.to_datetime(df['ts'], format='%Y-%m-%d %H:%M:%S')
    print(f'Converted timeseries')
    temp_2 = df.groupby(df["ts"].dt.hour).count()['agent'].values
    print(f'Group by')
    lists_of_lists = [map_hour, temp_2]
    temp = [sum(x) for x in zip(*lists_of_lists)]

for i in range(10, 20):
    data_key = f'part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    print(f'Reading {data_key}')
    df = pd.read_csv(data_location)
    print(f'Read {data_key}')
    df = df.dropna()
    print(f'Dropped Nones')
    df['ts'] = pd.to_datetime(df['ts'], format='%Y-%m-%d %H:%M:%S')
    print(f'Converted timeseries')
    temp_2 = df.groupby(df["ts"].dt.hour).count()['agent'].values
    print(f'Group by')
    lists_of_lists = [map_hour, temp_2]
    temp = [sum(x) for x in zip(*lists_of_lists)]

Reading part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00002-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00002-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00003-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00003-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00004-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00004-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00005-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00005-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Conv

In [16]:
max_value = max(temp)
print(max_value)
max_index = temp.index(max_value)
print(max_index)

620407.0
19


## **What is peak hour in terms of sales?**

**Proposed**: group by hour each transaction dataset > created (key, value) pair where key is hour and value is the aggregated value of the transactions > Aggregated all the counts based on the key for all transactions > Got the hour (key) with maximum aggregated sum of price.

**Assumption**: Each slice has consecutive samples

In [17]:
data_location = 's3://robi-datathon-dataset-2022/data/product.csv'

print(f'Reading {data_location}')
df_prod = pd.read_csv(data_location)
print(f'Read {data_location}')

Reading s3://robi-datathon-dataset-2022/data/product.csv
Read s3://robi-datathon-dataset-2022/data/product.csv


In [18]:
df_prod.head()

,price,dt,product
0,4.00,20220101,Product1
1,7.00,20220310,Product2
2,8.00,20220101,Product3
3,9.00,20220101,Product4
4,9.02,20220305,Product5


In [19]:
map_hour = [0.0]*24

for i in range(0, 10):
    data_key = f'part-0000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    print(f'Reading {data_key}')
    df = pd.read_csv(data_location)
    print(f'Read {data_key}')
    df = df.dropna()
    print(f'Dropped Nones')
    df = pd.merge(df, df_prod, on=['product'])
    df['ts'] = pd.to_datetime(df['ts'], format='%Y-%m-%d %H:%M:%S')
    print(f'Converted timeseries')
    temp_2 = df.groupby(df["ts"].dt.hour).sum()['price']
    print(f'Group by')
    lists_of_lists = [map_hour, temp_2]
    temp = [sum(x) for x in zip(*lists_of_lists)]

for i in range(10, 20):
    data_key = f'part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    print(f'Reading {data_key}')
    df = pd.read_csv(data_location)
    print(f'Read {data_key}')
    df = df.dropna()
    print(f'Dropped Nones')
    df = pd.merge(df, df_prod, on=['product'])
    df['ts'] = pd.to_datetime(df['ts'], format='%Y-%m-%d %H:%M:%S')
    print(f'Converted timeseries')
    temp_2 = df.groupby(df["ts"].dt.hour).sum()['price']
    print(f'Group by')
    lists_of_lists = [map_hour, temp_2]
    temp = [sum(x) for x in zip(*lists_of_lists)]

Reading part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00002-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00002-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00003-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00003-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00004-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00004-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Converted timeseries
Group by
Reading part-00005-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Read part-00005-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Dropped Nones
Conv

In [20]:
max_value = max(temp)
print(max_value)
max_index = temp.index(max_value)
print(max_index)

31492866.03
19


## **Which customer was travelling the most?**

**Proposed**: group by customer > sorted by timestamp > distance between consecutive samples > Sum-up all distance for each customer > save list of distance for customers(key) for each slice > merge the lists for all slices by summing up against keys > get highest distance.

**Assumption**: Each slice has consecutive samples

*Processes are done using multiprocessing nad numpy arrays for faster execution*

In [7]:
data_location = 's3://robi-datathon-dataset-2022/data/agent.csv'

print(f'Reading {data_location}')
df_agent = pd.read_csv(data_location)
print(f'Read {data_location}')

Reading s3://robi-datathon-dataset-2022/data/agent.csv
Read s3://robi-datathon-dataset-2022/data/agent.csv


In [8]:
df_agent.head()

,agent,lat,lon,ts
0,632791e6ac867e8257e6fccb2f5ef5eb8e6eed8ee2bd03...,23.541481,89.170306,2022-06-19 00:00:00
1,c48ff60fa46230c0ebbcfa1e7064224bda84952554f6ff...,23.486257,89.251074,2022-06-19 00:00:00
2,264c4537ccffb2b544ed7f4bac7d8cf04f08ccb23ea634...,23.563920,89.047684,2022-06-19 00:00:00
3,7a876f47bde543409b9e546ab128bcb10fe304dbd9cecb...,NaN,NaN,2022-06-19 00:00:00
4,5a226d44dec4bae00991d8e8d9ae370a77640e7e62734f...,23.553360,89.098775,2022-06-19 00:00:00


In [9]:
def sum_dist(df_x):
    sum = 0
    p1 = (0.0, 0.0)
    idx = 0
    for row in df_x.values:
        if idx == 0:
            # print("skipped")
            p1 = (row[5], row[6])
            idx = idx + 1
            continue
        
        p2 = (row[5], row[6])
        sum += distance.great_circle(p1, p2).km
        # print(f'Exec: {p1} {p2}: {sum}')
        p1 = p2
        idx = idx + 1
    return(sum)

In [10]:
param_list = []
for i in range(0, 10):
    data_key = f'part-0000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    param_list.append(data_location)
    
for i in range(10, 20):
    data_key = f'part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    param_list.append(data_location)
    
param_list

['s3://robi-datathon-dataset-2022/data/transaction/part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00002-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00003-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00004-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00005-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00006-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00007-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://robi-datathon-dataset-2022/data/transaction/part-00008-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv',
 's3://rob

In [11]:
def process_slices(data_location):
    print(f"Reading {data_location}")
    df = pd.read_csv(data_location)
    print(f'Read')
    df = df.dropna()
    print(f'Dropped Nones')
    df = pd.merge(df, df_agent, on=['agent'])
    temp_2 = df.groupby(df["customer"]).apply(sum_dist)
    print(f'Group by')
    return temp_2.to_dict()

In [ ]:
try:
    pool = Pool(8) # on 8 processors
    data_outputs = pool.map(process_slices, param_list)
finally: # To make sure processes are closed in the end, even if errors happen
    pool.close()
    pool.join()

Reading s3://robi-datathon-dataset-2022/data/transaction/part-00002-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Reading s3://robi-datathon-dataset-2022/data/transaction/part-00004-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Reading s3://robi-datathon-dataset-2022/data/transaction/part-00001-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Reading s3://robi-datathon-dataset-2022/data/transaction/part-00000-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Reading s3://robi-datathon-dataset-2022/data/transaction/part-00003-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Reading s3://robi-datathon-dataset-2022/data/transaction/part-00006-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Reading s3://robi-datathon-dataset-2022/data/transaction/part-00007-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv
Reading s3://robi-datathon-dataset-2022/data/transaction/part-00005-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv


In [ ]:
from collections import Counter
c = Counter()
for d in data_outputs:
    c.update(d)

In [ ]:
del data_outputs

In [ ]:
dict(c)

In [ ]:
c = dict(c)
Keymax = max(zip(c.values(), c.keys()))[1]
print(Keymax)

## **Draft**

In [26]:
result_squares.get(timeout=1)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [27]:
result_cubes.get(timeout=1)

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]

In [7]:
for i in range(1,10):
    data_key = f'part-0000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    df = pd.concat([df,pd.read_csv(data_location)])

In [8]:
for i in range(10,20):
    data_key = f'part-000{i}-afd80227-80b2-4b6a-aaaf-6e93851fc5cd-c000.csv'
    data_location = 's3://{}/{}'.format(bucket, data_key)
    df = pd.concat([df,pd.read_csv(data_location)])

In [9]:
df.shape

(98266017, 5)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98266017 entries, 0 to 4913306
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   agent     object
 1   customer  object
 2   product   object
 3   dt        int64 
 4   ts        object
dtypes: int64(1), object(4)
memory usage: 4.4+ GB


In [ ]:
df_agent = pd.read_csv('s3://{}/{}'.format(bucket, data_key))